In [1]:
!ls /gpfs/data/oermannlab/users/alyaka01/checkpoints/obsidian/

collated	 stage_2_05_01		    stage_3_gpt_01_03_05
not_interesting  stage_2_05_03		    stage_3_gpt_01_03_10
stage_1_01	 stage_2_05_05		    stage_3_gpt_01_03_20_fake
stage_1_03	 stage_2_05_10		    stage_3_gpt_01_05_05
stage_1_05	 stage_3_both_01_03_05	    stage_3_gpt_01_05_10
stage_2_00_01	 stage_3_both_01_03_10	    stage_3_gpt_01_10_10
stage_2_00_03	 stage_3_both_05_10_05	    stage_3_gpt_03_03_03
stage_2_00_05	 stage_3_both_05_10_10	    stage_3_gpt_03_03_05
stage_2_01_01	 stage_3_both_fix_01_03_05  stage_3_gpt_03_03_10
stage_2_01_03	 stage_3_both_fix_01_03_10  stage_3_gpt_03_05_05
stage_2_01_05	 stage_3_both_fix_05_10_05  stage_3_gpt_03_05_10
stage_2_01_10	 stage_3_both_fix_05_10_10  stage_3_gpt_03_10_10
stage_2_03_01	 stage_3_gpt_00_00_03	    stage_3_gpt_05_03_03
stage_2_03_03	 stage_3_gpt_00_03_03	    stage_3_gpt_05_05_05
stage_2_03_05	 stage_3_gpt_01_00_03	    stage_3_gpt_05_10_10
stage_2_03_10	 stage_3_gpt_01_03_03


In [ ]:
import torch
from transformers import AutoConfig, AutoTokenizer
from transformers import LlavaNextForConditionalGeneration, LlavaNextProcessor
from huggingface_hub import login

# Login with your token
login(token="")

hf_snipe = "llava-hf/llava-v1.6-34b-hf"
config = AutoConfig.from_pretrained(hf_snipe)

# 1. Build the base model in bf16
model = LlavaNextForConditionalGeneration.from_pretrained(
    hf_snipe,
    config=config,
    torch_dtype=torch.bfloat16,
)

# 2. Load your bf16 checkpoint
checkpoint = torch.load("/gpfs/data/oermannlab/users/alyaka01/checkpoints/obsidian/stage_2_01_03/checkpoint_2.pt", 
                        map_location="cpu")
state_dict = checkpoint.get("model_state_dict", checkpoint)

model.load_state_dict(state_dict, strict=True)

# 3. Make sure HF metadata says bf16
model.config.torch_dtype = "bfloat16"

# 4. Push in bf16
repo_id = "NYU-OLAB/LLaVA-Next-Med-OLAB"
model.push_to_hub(repo_id, commit_message="Upload bf16 weights")

processor = LlavaNextProcessor.from_pretrained(hf_snipe)
processor.push_to_hub("NYU-OLAB/LLaVA-Next-Med-OLAB")